In [2]:
import numpy as np
import pickle
import sys
from typing import List
import torch
import os

In [3]:
participant_data_path = './EEG/number'
processed_data_path = './EEG/processed_data'
split_data_path = './EEG/split_data'

In [4]:
ps = os.listdir(participant_data_path)
participants=[]
for p in ps:
    p= p[:3]
    if p not in participants:
        participants.append(p)
        
participants= sorted(participants)
# participants, len(participants)

In [5]:
load_name=f"userfold_data_scaled_p_dictionary-number"
data_dir = "./EEG/split_data/standard_scaled"
try:
    raw_user_fold= pickle.load(open(os.path.join(data_dir, f"{load_name}.pkl"), "rb"))
except:
    print(f"pickle file does not exist. Use EEG-Preprocess.ipynb and EEG-Split.ipynb to save data setting.")
    sys.exit()

In [6]:
from utilities.userfold_framework import *
from utilities.EEG_func import *
import Models.model_func as Model_Func
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List
from torcheeg.models import EEGNet
# import Models.model_func as Model_Func
from torch import nn
from Models.multi_models import *

DEVICE= torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

learning_rate = 0.00005
batch_size = 64
n_epochs = 300
transpose_channels=True
participants_dictionary=[]
# participants_online_dictionary=[]
participants_grads_dictionary={}
b_acc_list=[]
c0_acc_list=[]
c1_acc_list=[]

# EPOCH=[
    
# ]

for i in [0]:#range(len(participants)):

    train_dataloader, val_dataloader, classes, input_dim, class_ratio= user_fold_load(i,
                                                                                      raw_user_fold,
                                                                                      participants,
                                                                                      batch_size=batch_size,
                                                                                      transpose_channels=transpose_channels)

    classifier= EEGNet(
        chunk_size=input_dim[1],
        num_electrodes=input_dim[0],
        num_classes=classes,
        kernel_1= 32,
        kernel_2=32,
        F1=8,
        F2=16,
        dropout=0.5
    ).to(DEVICE)
    
#     resnet = ResNetPlus(input_dim[0], classes, bn_1st=False)
#     softmax_activation = nn.LogSoftmax(dim=1)
#     classifier = nn.Sequential(resnet, softmax_activation).to(DEVICE)

    
#     classifier = DataGliderBasic_Model(DEVICE, input_dim, classes)
#     classifier.to(DEVICE)
    
#     optimizer= torch.optim.RMSprop(classifier.parameters(), lr=learning_rate)
    
    
    criterion= torch.nn.CrossEntropyLoss(weight=torch.tensor(class_ratio, dtype=torch.float).to(DEVICE))
#     criterion = nn.NLLLoss(weight=torch.tensor(class_ratio, dtype=torch.float).to(DEVICE))
        
    saved_dir= "./EEG/saved_models/Userfold/run0"
    model = EEGNet_IE_TS_Wrapper(DEVICE, classifier, input_dim[1]).to(DEVICE)
    optimizer = torch.optim.Adam(classifier.parameters(), lr = learning_rate)
    scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2)

    
    train_func= eeg_grad_train
    model.training_procedure(iteration=n_epochs,
                                    train_dataloader=train_dataloader,
                                     val_dataloader=val_dataloader,
                                     print_cycle=2,
                                     path=f"./dictionary/intermdiate_dicts",
                                     loss_func=criterion,
                                     optimiser=optimizer, #scheduler=scheduler,
                                     train_func=train_func
                                    )
    if model.epoch == n_epochs+1:
        EPOCH= n_epochs
    else:
        EPOCH= model.epoch
    
#     torch.save(model.state_dict(), 
#            os.path.join(
#                saved_dir, f"Userfold-{participants[i]}-EEGNet-Grad_TS-e{EPOCH}.pt"
#            )
#     )

#     pickle.dump( model.return_IE_grad(), 
#                 open(f"{saved_dir}/Userfold-{participants[i]}-EEGNet-Grad_TS-grad_all-e{EPOCH}.pkl", "wb") 
#                )    

# OR
#     model.load_state_dict(
#     torch.load(
#         open(
#             os.path.join(
#                 saved_dir, f"Userfold-{participants[i]}-EEGNet-Grad_TS-e{n_epochs}.pt"
#             ), "rb"
#         )
#               )
#     )
#     pickle.load( 
#                 open(f"{saved_dir}/Userfold-{participants[i]}-EEGNet-Grad_TS-w-e{EPOCH}.pkl", "rb") 
#                )  
    
    prediction, dictionary= model.prediction_procedure(val_dataloader, dict_flag=True)
    
    ys= np.concatenate([y.detach().cpu().numpy() for x, y in val_dataloader])
    
    c0_acc, c1_acc, b_acc= calculate_accuracy(ys, prediction)
    print("c0_acc", c0_acc, ", c1_acc", c1_acc, ", b_acc", b_acc)
    b_acc_list.append(b_acc)
    c0_acc_list.append(c0_acc)
    c1_acc_list.append(c1_acc)
    participants_dictionary.append(dictionary)
    

tmp=[]
for i, dictionary in enumerate(participants_dictionary):
    print(f"User {participants[i]} f1: {dictionary['weighted avg']['f1-score']} acc: {dictionary['accuracy']}")
    print(f" c0: {c0_acc_list[i]} c1: {c1_acc_list[i]} bacc: {b_acc_list[i]}")
    tmp.append(dictionary['weighted avg']['f1-score'])

print(f"average {np.mean(tmp)}")
print()
print(np.array(b_acc_list).mean())
print(np.array(c1_acc_list).mean())
print(np.array(c0_acc_list).mean())


Iterations:   0%|                                           | 0/300 [00:00<?, ?it/s]

Epoch  0 , loss 0.6515877702656914


Iterations:   0%|                                   | 1/300 [00:03<15:32,  3.12s/it]

Epoch:  0
t_loss:  0.6515877702656914 , v_loss:  0.6893738508224487
t_acc:  0.5633364456893869 , v_acc:  0.639751552795031
t_recall:  0.4929070421761985 , v_recall:  0.5134234234234234
t_prec:  0.493129218900675 , v_prec:  0.5212250712250712
t_f:  0.4928268095833055 , v_f:  0.5005348737697903
////////


Iterations:   1%|▏                                  | 2/300 [00:03<08:21,  1.68s/it]

...

Iterations: 100%|████████████████████████████████▉| 299/300 [04:25<00:00,  1.11it/s]

Epoch:  298
t_loss:  0.4074268078102785 , v_loss:  0.49157528579235077
t_acc:  0.711484593837535 , v_acc:  0.8012422360248447
t_recall:  0.5338370321571211 , v_recall:  0.6964864864864865
t_prec:  0.7266409266409266 , v_prec:  0.8316605839416058
t_f:  0.4852731432588683 , v_f:  0.7192676547515258
////////


Iterations: 100%|█████████████████████████████████| 300/300 [04:25<00:00,  1.13it/s]

Epoch  299 , loss 0.3966907628026663


108 21

c0_acc 0.972972972972973 , c1_acc 0.42 , b_acc 0.6964864864864865
User S01 f1: 0.7767440852994849 acc: 0.8012422360248447
 c0: 0.972972972972973 c1: 0.42 bacc: 0.6964864864864865
average 0.7767440852994849

0.6964864864864865
0.42
0.972972972972973


In [13]:
userfold_results_summary(participants_dictionary, participants)
userfold_classwise_results_summary(participants_dictionary, participants)


IndexError: list index out of range

In [8]:
model.return_IE_weights()

array([1.0195702 , 0.98991334, 0.98931396, 1.0119187 , 0.9500421 ,
       1.0051342 , 1.0161706 , 0.97315997, 1.0279183 , 1.0489782 ,
       1.004604  , 0.9855444 , 1.0191396 , 0.98882616, 1.0202456 ,
       1.0420084 , 0.998871  , 1.0053743 , 0.9608169 , 1.0027977 ,
       0.97733986, 1.0049847 , 0.99244595, 0.992699  , 0.98052275,
       1.0038934 , 1.0224276 , 0.9800595 , 0.9673722 , 0.97339386,
       1.0022808 , 1.0092655 ], dtype=float32)